In [29]:
from pycoingecko import CoinGeckoAPI
from pymongo import MongoClient
import pandas as pd
import time
import datetime

class BitcoinAvg():
    
    def __init__(self, host = 'mongodb://localhost', from_day = "01-01-2022", to_day = "31-03-2022", _id = '1_4-2022', avg = '5'):
        self.host = host
        self.from_day = from_day
        self.to_day = to_day
        self._id = _id
        self.avg = avg
        
    def connection(self, collection):
        cl = MongoClient(self.host)
        db = cl['Ktest']
        coll = db[collection]
        self.cl = cl
        return coll
    
    def close_connection(self):
        self.cl.close()
    

    def get_prices(self):
        print('Getting prices from:', self.from_day,'to',self.to_day,'...')
        from_day = time.mktime(datetime.datetime.strptime(self.from_day, "%d-%m-%Y").timetuple())
        to_day = time.mktime(datetime.datetime.strptime(self.to_day, "%d-%m-%Y").timetuple())
        
        cg = CoinGeckoAPI()
        prices = cg.get_coin_market_chart_range_by_id(id='bitcoin', from_timestamp=from_day, to_timestamp=to_day, vs_currency='usd')['prices']
        prices = pd.DataFrame(prices, columns=['timestamp', 'price'])
        prices.timestamp = prices.timestamp.astype(dtype='datetime64[ms]') 
        prices = prices.sort_values(by='timestamp').reset_index(drop=True)
        prices['date'] = prices['timestamp'].dt.strftime('%Y-%m-%d')
        prices = prices.drop_duplicates(subset='date', keep='last')[['date','price']]
        
        print('Loading prices to mongodb...')
        json_prices = prices.set_index('date')['price'].to_dict()
        json_prices['_id']=self._id
        
        coll = self.connection('bitcoin')
        coll.insert_one(json_prices)
        self.close_connection()
        
        print('Prices saved with id =' , self._id)
        
        
    def get_avg_prices(self):
        
        print('Getting prices from _id=',self._id, 'from mongodb')
        coll = self.connection('bitcoin')
        data = coll.find_one(self._id)
        self.close_connection()
        
        del data["_id"]
        df = pd.DataFrame.from_dict(data, orient='index', columns=['price'])
        df = df.reset_index().rename(columns={'index': 'date'})
        df.date = df.date.astype(dtype='datetime64[ms]')
        df = df.set_index('date').resample('5D').mean().reset_index()
        
        df.date = df.date.dt.strftime('%Y-%m-%d')
        json_prices_avg = df.set_index('date')['price'].to_dict()
        json_prices_avg['_id']=self._id
        
        print('Loading averange prices to mongodb...')
        coll = self.connection(f'bitcoin_avg{self.avg}')
        coll.insert_one(json_prices_avg)
        self.close_connection()
        print('Averange prices saved in _id=', self._id)
        
    def main(self):
        
        self.get_prices()
        self.get_avg_prices()
        
    

In [30]:
function = BitcoinAvg(from_day = "01-01-2021", to_day = "31-03-2021", _id = '1_4-2021')

In [31]:
function.main()

Getting prices from: 01-01-2021 to 31-03-2021 ...
Loading prices to mongodb...
Prices saved with id = 1_4-2021
Getting prices from _id= 1_4-2021 from mongodb
Loading averange prices to mongodb...
Averange prices saved in _id= 1_4-2021
